In [20]:
from __future__ import absolute_import
from __future__ import division, unicode_literals

# Get the python version (used to try decode an unknow instance to unicode)
from sys import version_info

PY3 = version_info[0] == 3

# Use classical Snowball stemmer for english
import nltk
from nltk.util import ngrams

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
# class WordTokenizer(object):
# 	"""docstring for WordTokenizer"""
# 	def __init__(self):
# 		pass
# 	def tokenize(self, arg):
# 		return nltk.word_tokenize(arg)
# tokenizer =  WordTokenizer()



from nltk.corpus import stopwords
stopset = frozenset(stopwords.words('english'))

# Convert an object to its unicode representation (if possible)
def to_unicode(object):
	if isinstance(object, str):
		return object
	elif isinstance(object, bytes):
		return object.decode("utf8")
	else:
		print (object)
		if PY3:
			if hasattr(instance, "__str__"):
				return unicode(instance)
			elif hasattr(instance, "__bytes__"):
				return bytes(instance).decode("utf8")
		else:
			if hasattr(instance, "__unicode__"):
				return unicode(instance)
			elif hasattr(instance, "__str__"):
				return bytes(instance).decode("utf8")

# normalize and stem the word
def stem_word(word):
	return stemmer.stem(normalize_word(word))

# convert to unicode and convert to lower case
def normalize_word(word):
	return to_unicode(word).lower()

# convert the sentence to a list of tokens
def sentence_tokenizer(sentence):
	return tokenizer.tokenize(sentence)

def get_len(element):
	return len(tokenizer.tokenize(element))

def get_ngrams(sentence, N):
	tokens = tokenizer.tokenize(sentence.lower())
	clean = [stemmer.stem(token) for token in tokens]
	return [gram for gram in ngrams(clean, N)]

In [21]:
from nlp_utils import *

import math
import numpy as np

from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

def get_all_content_words_lemmatized(sentences, N=1):
	all_words = []
	for s in sentences:
		all_words.extend([wordnet_lemmatizer.lemmatize(r) for r in tokenizer.tokenize(s)])
	if N == 1:
		content_words = [w for w in all_words if w not in stopset]
	normalized_content_words = map(normalize_word, content_words)
	if N > 1:
		return [gram for gram in ngrams(normalized_content_words, N)]
	return normalized_content_words

def get_all_content_words_stemmed(sentences, N=1):
	def is_ngram_content(g):
		for a in g:
			if not(a in stopset):
				return True
		return False

	all_words = []
	for s in sentences:
		all_words.extend([stemmer.stem(r) for r in tokenizer.tokenize(s)])

	if N == 1:
		content_words = [w for w in all_words if w not in stopset]
	else:
		content_words = all_words

	normalized_content_words = map(normalize_word, content_words)
	if N > 1:
		return [gram for gram in ngrams(normalized_content_words, N) if is_ngram_content(gram)]
	return normalized_content_words

def get_all_content_words(sentences, N=1):
	all_words = []
	for s in sentences:
		all_words.extend(tokenizer.tokenize(s))
	content_words = [w for w in all_words if w not in stopset]
	normalized_content_words = map(normalize_word, content_words)
	if N > 1:
		return [gram for gram in ngrams(normalized_content_words, N)]
	return normalized_content_words

def get_content_words_in_sentence(sentence):
	words = tokenizer.tokenize(sentence)
	return [w for w in words if w not in stopset]

def compute_tf_doc(docs, N=1):
	sentences = []
	for title, doc in docs:
		sentences.append(title)
		sentences.extend(doc)

	content_words = list(set(get_all_content_words_stemmed(sentences, N)))
	docs_words = []
	for title, doc in docs:
		s_tmp = [title]
		s_tmp.extend(doc)
		docs_words.append(get_all_content_words_stemmed(s_tmp, N))

	word_freq = {}
	for w in content_words:
		w_score = 0
		for d in docs_words:
			if w in d:
				w_score += 1
		if w_score != 0:
			word_freq[w] = w_score

	content_word_tf = dict((w, f / float(len(word_freq.keys()))) for w, f in word_freq.items())
	return content_word_tf

def compute_word_freq(words):
	word_freq = {}
	for w in words:
		word_freq[w] = word_freq.get(w, 0) + 1
	return word_freq

def compute_tf(sentences, N=1):
	content_words = get_all_content_words_stemmed(sentences, N)
	content_words_count = len(list(content_words))
	content_words_freq = compute_word_freq(content_words)

	content_word_tf = dict((w, f / float(content_words_count)) for w, f in content_words_freq.items())
	return content_word_tf

def compute_average_freq(l_freq_1, l_freq_2):
	average_freq = {}

	keys = set(l_freq_1.keys()) | set(l_freq_2.keys())

	for k in keys:
		s_1 = l_freq_1.get(k, 0)
		s_2 = l_freq_2.get(k, 0)

		average_freq[k] = (s_1 + s_2) / 2.

	return average_freq

def kl_divergence(summary_freq, doc_freq):
	sum_val = 0
	for w, f in summary_freq.items():
		if w in doc_freq:
			sum_val += f * math.log(f / float(doc_freq[w]))
	
	return sum_val

def js_divergence(sys_summary, doc_freq):
	summary_freq = compute_tf(sys_summary)
	average_freq = compute_average_freq(summary_freq, doc_freq)

	jsd = kl_divergence(summary_freq, average_freq) + kl_divergence(doc_freq, average_freq)
	return jsd / 2.

In [22]:
from nlp_utils import *

def get_len(element):
	return len(tokenizer.tokenize(element))
	
def greedy_optimizer(sorted_elements, K):
	available_space = K
	summary = []

	while True:
		for element in sorted_elements:
			len_element = get_len(element[0])
			if available_space - len_element >= 0:
				idx = sorted_elements.index(element)
				del sorted_elements[idx]
				available_space -= len_element
				summary.append(element[0])
		else:
			break

	return summary


In [23]:
import numpy as np
import random
from copy import deepcopy

#from greedy import greedy_optimizer

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

class GeneticOptimizer(object):
	def __init__(self, fitness_fun, docs, docs_representation, max_length, population_size, survival_rate, mutation_rate, reproduction_rate, maximization=False, sentences_rep=None):
		np.random.seed(123)

		self._fitness_fun = fitness_fun
		self._population_size = population_size
		self._survival_rate = survival_rate
		self._mutation_rate = mutation_rate
		self._reproduction_rate = reproduction_rate
		self._maximization = maximization

		self._docs = docs
		self._docs_representation = docs_representation
		self._sentences_rep = sentences_rep
		self._max_length = max_length
		
		self._sentences = []
		self._sentence_tokens = []
		for title, doc in docs:
			self._sentences.append(title)
			self._sentence_tokens.append(tokenizer.tokenize(title))
			self._sentences.extend(doc)
			for s in doc:
				self._sentence_tokens.append(tokenizer.tokenize(s))

	def _create_random_individual(self):
		random_scores = np.random.rand(len(self._sentences))
		scored_sentences = zip(self._sentences, random_scores)
		sorted_sentences = sorted(scored_sentences, key=lambda tup: tup[1], reverse=True)
		return greedy_optimizer(sorted_sentences, self._max_length)

	def _generate_random_population(self, n):
		population = []
		for i in range(n):
			population.append(self._create_random_individual())
		return population

	def _score_population(self, population):
		scored_population = []
		for individual in population:
			# score = self._fitness_fun(individual, self._docs)
			if self._sentences_rep != None:
				score = self._fitness_fun(individual, self._docs_representation, self._sentences_rep)
			else:
				score = self._fitness_fun(individual, self._docs_representation)
			scored_population.append((individual, score))

		return scored_population

	def _select_survivors(self, scored_population):
		sorted_population = sorted(scored_population, key=lambda tup: tup[1], reverse=self._maximization)

		percentage_winner = 0.5

		to_keep = int(self._survival_rate * self._population_size)
		number_winners = int(percentage_winner * to_keep)
		winners = [tup[0] for tup in sorted_population[:number_winners]]
		
		losers = sorted_population[number_winners:]

		number_losers = int((1 - percentage_winner) * to_keep) 

		survivors = deepcopy(winners)
		random_scores = np.random.rand(len(losers))

		sorted_losers = sorted(zip(losers, random_scores), key=lambda tup: tup[1])
		loser_survivors = [tup[0][0] for tup in sorted_losers[:number_losers]]

		survivors.extend(loser_survivors)
		return survivors, winners

	def _new_generation(self, scored_population):
		new_generation, winners = self._select_survivors(scored_population)
		new_generation = self._mutate(new_generation)
		new_generation.extend(self._reproduction(winners, len(new_generation)))
		individuals_to_create = self._population_size - len(new_generation)
		new_generation.extend(self._generate_random_population(individuals_to_create))
		
		return new_generation

	def _len_individual(self, individual):
		len_ = 0
		for sentence in individual:
			len_ += len(tokenizer.tokenize(sentence))
		return len_

	def _mutate(self, population, mutation_rate="auto"):
		if mutation_rate == "auto":
			mutation_rate = self._mutation_rate

		nb_mutant = int(mutation_rate * len(population))

		random_scores = np.random.rand(len(population))
		sorted_population = sorted(zip(population, random_scores), key=lambda tup: tup[1])
		mutants = [tup[0] for tup in sorted_population[:nb_mutant]]

		mutated = []
		i = 0
		for mutant in mutants:
			to_mutate = deepcopy(mutant)

			sentence_to_remove = random.choice(to_mutate)
			idx = to_mutate.index(sentence_to_remove)
			del to_mutate[idx]

			available_size = self._max_length - self._len_individual(to_mutate)
		
			available_sentences = [s[0] for s in zip(self._sentences, self._sentence_tokens) if len(s[1]) <= available_size]
			if available_sentences != []:
				i += 1
				sentence_to_add = random.choice(available_sentences)
				to_mutate.append(sentence_to_add)
				
				mutated.append(to_mutate)
		
		population.extend(mutated)
		return population

	def _reproduction(self, population_winners, population_size, reproduction_rate="auto"):
		if reproduction_rate == "auto":
			reproduction_rate = self._reproduction_rate

		parents = []
		number_families = int(reproduction_rate * population_size)
	
		for i in range(number_families):
			parents.append(random.sample(population_winners, 2))

		children = []
		for father, mother in parents:
			genetic_pool = [s for s in self._sentences if s in father]
			genetic_pool.extend([s for s in self._sentences if s in mother])
			random_scores = np.random.rand(len(genetic_pool))
			scored_sentences = zip(self._sentences, random_scores)
			sorted_sentences = sorted(scored_sentences, key=lambda tup: tup[1], reverse=True)
			child = greedy_optimizer(sorted_sentences, self._max_length)

			children.append(child)

		return children

	def initial_population(self):
		initial_population = self._generate_random_population(self._population_size)
		print ("initial population len:", len(initial_population))
		return initial_population

	def _is_better(self, scored_individual, best_scored_individual):
		if self._maximization:
			return scored_individual[1] > best_scored_individual[1]
		return scored_individual[1] < best_scored_individual[1]

	def evolve(self, epoch):
		population = self.initial_population()
		if self._maximization:
			best_individual = (None, -10000)
		else:
			best_individual = (None, 10000)
		for i in range(epoch):
			print ("epoch: ", i, " -- best individual: ", best_individual)
			scored_population = self._score_population(population)
			sorted_population = sorted(scored_population, key=lambda tup: tup[1], reverse=self._maximization)
			best_individual_in_generation = sorted_population[0]

			if self._is_better(best_individual_in_generation, best_individual):
				best_individual = best_individual_in_generation

			population = self._new_generation(scored_population)
		
		return best_individual

In [24]:
import numpy as np
import random
from copy import deepcopy

#from greedy import greedy_optimizer

#from JS import js_divergence

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

class SwarmOptimizer(object):
	def __init__(self, fitness_fun, docs,  docs_representation, max_length, number_locations, trial_limit, mfe, args=None, maximization=False):
		np.random.seed(123)

		self._fitness_fun = fitness_fun
		self._args = args
		self._maximization = maximization

		self._number_locations = number_locations
		self._trial_limit = trial_limit
		self._mfe = mfe

		self._docs = docs
		self._docs_representation = docs_representation
		self._max_length = max_length

		self._sentences = []
		self._sentence_tokens = []
		for title, doc in docs:
			self._sentences.append(title)
			self._sentence_tokens.append(tokenizer.tokenize(title))
			self._sentences.extend(doc)
			for s in doc:
				self._sentence_tokens.append(tokenizer.tokenize(s))


	def _create_random_food_location(self):
		random_scores = np.random.rand(len(self._sentences))
		scored_sentences = zip(self._sentences, random_scores)
		sorted_sentences = sorted(scored_sentences, key=lambda tup: tup[1], reverse=True)
		return greedy_optimizer(sorted_sentences, self._max_length)

	def _generate_random_foods(self, n):
		foods = []
		for i in range(n):
			foods.append(self._create_random_food_location())
		return foods

	def _len_location(self, location):
		len_ = 0
		for sentence in location:
			len_ += len(tokenizer.tokenize(sentence))
		return len_

	def _score_food_location(self, food_location):
		if self._args:
			return self._fitness_fun(food_location, self._docs_representation, self._args)
		return self._fitness_fun(food_location, self._docs_representation)

	def _random_local_search(self, food_location_old):
		food_location = deepcopy(food_location_old)
		sentence_to_remove = random.choice(food_location)
		idx = food_location.index(sentence_to_remove)
		del food_location[idx]

		available_size = self._max_length - self._len_location(food_location)
		
		available_sentences = [s[0] for s in zip(self._sentences, self._sentence_tokens) if len(s[1]) <= available_size]
		if available_sentences != []:
			sentence_to_add = random.choice(available_sentences)
			food_location.append(sentence_to_add)
				
		return food_location

	def initial_foods(self):
		initial_foods = self._generate_random_foods(self._number_locations)
		print ("initial number of foods :", len(initial_foods))
		return initial_foods

	def _is_better(self, score_a, score_b):
		if self._maximization:
			return score_a > score_b
		return score_a < score_b

	def swarm_disperse(self):
		foods_locations = self.initial_foods()
		score_vector = [self._score_food_location(loc) for loc in foods_locations]
		trials = [0] * len(foods_locations)
		number_fitness_evalutation = len(foods_locations)

		size_10_percent = int(0.1 * len(foods_locations))
		if self._maximization:
			best_location = (None, -10000)
		else:
			best_location = (None, 10000)

		epoch = 0
		while True:
			epoch += 1
			print ("epoch: ", epoch, " -- best location: ", best_location)

			# Employed Bees Phase
			for i, location in enumerate(foods_locations):
				new_location = self._random_local_search(location)
				score = self._score_food_location(new_location)
				number_fitness_evalutation += 1

				if self._is_better(score, score_vector[i]):
					if self._is_better(score, best_location[1]):
						best_location = (new_location, score)

					score_vector[i] = score
					foods_locations[i] = new_location
					trials[i] = 0
				else:
					trials[i] += 1

				if number_fitness_evalutation >= self._mfe:
					return best_location

			sum_ = sum(score_vector)
            
			if sum_ == 0:
				probability_vector = [100 for s in score_vector]
			else:
				probability_vector = [float(s) / float(sum_) for s in score_vector]

			# Onlooker Bees Phase
			s = 0
			t = 1
			while t < self._number_locations:
				r = random.uniform(0, 1)
				if r < probability_vector[s]:
					t += 1
					location = foods_locations[s]
					new_location = self._random_local_search(location)
					score = self._score_food_location(new_location)
					number_fitness_evalutation += 1

					if self._is_better(score, score_vector[i]):
						if self._is_better(score, best_location[1]):
							best_location = (new_location, score)

						score_vector[i] = score
						foods_locations[i] = new_location
						trials[i] = 0
					else:
						trials[i] += 1

					if number_fitness_evalutation >= self._mfe:
						return best_location
				s += 1
				if s == self._number_locations:
					s = 0

			# Scout Bees Phase
			mi = trials.index(max(trials))
			if trials[mi] > self._trial_limit:
				new_location = self._create_random_food_location()
				score = self._score_food_location(new_location)
				number_fitness_evalutation += 1
				trials[mi] = 0

				if self._is_better(score, best_location[1]):
					best_location = (new_location, score)

				if number_fitness_evalutation >= self._mfe:
					return best_location

		return best_location

In [25]:
def JS_Gen(docs, length_max, epoch, population_size=1000):
	sentences = []
	for title, doc in docs:
		sentences.append(title)
		sentences.extend(doc)

	doc_freq = compute_tf(sentences)
	
	gen_optimizer = GeneticOptimizer(fitness_fun=js_divergence, 
									 docs=docs, 
									 docs_representation = doc_freq,
									 max_length=length_max, 
									 population_size=population_size, 
									 survival_rate=0.4,
									 mutation_rate=0.2,
									 reproduction_rate=0.4,  
									 maximization=False)

	return gen_optimizer.evolve(epoch)

def JS_Swarm(docs, length_max, mfe=80000, number_locations=1000):
	sentences = []
	for title, doc in docs:
		sentences.append(title)
		sentences.extend(doc)

	doc_freq = compute_tf(sentences)
	
	swarm_optimizer = SwarmOptimizer(fitness_fun=js_divergence, 
									 docs=docs, 
									 docs_representation = doc_freq,
									 max_length=length_max, 
									 number_locations=number_locations, 
									 trial_limit=400,
									 mfe=mfe,  
									 maximization=False)

	return swarm_optimizer.swarm_disperse()

if __name__ == '__main__':
	doc_1 = ("title of the first document", ["first sentence of first doc", "second sentence", "third sentence of the first document here", "another one", "what is going on "])
	doc_2 = ("second title", ["one sentnece quite random", "here is another one completely random", "sentence here", "que pasa", "a sentence in an other document"])
	doc_3 = ("title of the third document", ["it will be a short docuemnt", "only two sentences"])
	docs = [doc_1, doc_2, doc_3]

	length_max = 10
	epoch = 20
	population_size = 10
	print ("Genetic Algorithm example:")
	print (JS_Gen(docs, length_max, epoch, population_size))

	print ("\n==================\n")
	mfe = 400
	number_locations = population_size
	print ("Swarm Intelligence example:")
	print (JS_Swarm(docs, length_max, mfe, number_locations))

Genetic Algorithm example:
initial population len: 10
epoch:  0  -- best individual:  (None, 10000)
epoch:  1  -- best individual:  (['second title', 'another one', 'one sentnece quite random', 'sentence here'], 0.0)
epoch:  2  -- best individual:  (['second title', 'another one', 'one sentnece quite random', 'sentence here'], 0.0)
epoch:  3  -- best individual:  (['second title', 'another one', 'one sentnece quite random', 'sentence here'], 0.0)
epoch:  4  -- best individual:  (['second title', 'another one', 'one sentnece quite random', 'sentence here'], 0.0)
epoch:  5  -- best individual:  (['second title', 'another one', 'one sentnece quite random', 'sentence here'], 0.0)
epoch:  6  -- best individual:  (['second title', 'another one', 'one sentnece quite random', 'sentence here'], 0.0)
epoch:  7  -- best individual:  (['second title', 'another one', 'one sentnece quite random', 'sentence here'], 0.0)
epoch:  8  -- best individual:  (['second title', 'another one', 'one sentnece qu